<a href="https://colab.research.google.com/github/jeremysb1/predictive-analytics/blob/main/rfm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Directories, Library, and Data

In [13]:
%cd /content/drive/MyDrive/Business Analyst Course/Segmentation/RFM

/content/drive/MyDrive/Business Analyst Course/Segmentation/RFM


In [14]:
#import libraries
import pandas as pd
from datetime import timedelta
import matplotlib.pyplot as plt

In [15]:
#data
data = pd.read_csv("online_shoppers.csv", encoding = "ISO-8859-1").dropna()
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


#Data Preparation

In [16]:
#creating sales column
data['sales'] = data['Quantity'] * data['UnitPrice']
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,sales
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34


In [17]:
#transform date variable
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

#get last date variable
snapshot_date = data['InvoiceDate'].max() + timedelta(days = 1)
snapshot_date

Timestamp('2011-12-10 12:50:00')

In [18]:
#Aggregate on Customer Level
df = data.groupby(['CustomerID']).agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,
    'InvoiceNo': 'count',
    'sales': 'sum'
})
df.head()

,InvoiceDate,InvoiceNo,sales
CustomerID,,,
12346.0,326,2,0.00
12347.0,2,182,4310.00
12348.0,75,31,1797.24
12349.0,19,73,1757.55
12350.0,310,17,334.40


In [19]:
#Create basket / Monetary Variable
df['monetary'] = df.sales / df.InvoiceNo
df.head(2)

,InvoiceDate,InvoiceNo,sales,monetary
CustomerID,,,,
12346.0,326,2,0.0,0.000000
12347.0,2,182,4310.0,23.681319


In [20]:
#Dropping sales variable
df = df.drop(columns = 'sales')
df.head()

,InvoiceDate,InvoiceNo,monetary
CustomerID,,,
12346.0,326,2,0.000000
12347.0,2,182,23.681319
12348.0,75,31,57.975484
12349.0,19,73,24.076027
12350.0,310,17,19.670588


In [21]:
#changing variable names
df.rename(columns = {'InvoiceDate': 'recency',
                     'InvoiceNo': 'frequency'}, inplace = True)
df.head(1)

,recency,frequency,monetary
CustomerID,,,
12346.0,326,2,0.0


#RFM Preparation

In [26]:
#create frequency groups
df['F'] = pd.qcut(x = df['frequency'], q = 4, labels = range(1, 5, 1))
df['M'] = pd.qcut(x = df['monetary'], q = 4, labels = range(1, 5, 1))
df['R'] = pd.qcut(x = df['recency'], q = 4, labels = range(4, 0, -1))
df.head(5)

,recency,frequency,monetary,F,M,R
CustomerID,,,,,,
12346.0,326,2,0.000000,1,1,1
12347.0,2,182,23.681319,4,4,4
12348.0,75,31,57.975484,2,4,2
12349.0,19,73,24.076027,3,4,3
12350.0,310,17,19.670588,1,3,1


In [27]:
#RFM Score
df['RFM'] = df[['R', 'F', 'M']].sum(axis = 1)
df.head(2)

,recency,frequency,monetary,F,M,R,RFM
CustomerID,,,,,,,
12346.0,326,2,0.000000,1,1,1,3
12347.0,2,182,23.681319,4,4,4,12


In [28]:
#Create the RFM Function
def rfm_segment(df):
  if df['RFM'] >= 11:
    return 'SuperStar'
  elif ((df['RFM'] >=8) and (df['RFM'] < 11)):
    return 'Future Champion'
  elif ((df['RFM'] >=6) and (df['RFM'] < 8)):
    return 'High Potential'
  else:
    return 'Low Relevance'

#RFM

In [29]:
#Apply RFM Function
df['RFM_level'] = df.apply(rfm_segment, axis = 1)
df.tail()

,recency,frequency,monetary,F,M,R,RFM,RFM_level
CustomerID,,,,,,,,
18280.0,278,10,18.060000,1,3,1,5,Low Relevance
18281.0,181,7,11.545714,1,2,1,4,Low Relevance
18282.0,8,13,13.584615,1,2,4,7,High Potential
18283.0,4,756,2.771005,4,1,4,9,Future Champion
18287.0,43,70,26.246857,3,4,3,10,Future Champion


In [30]:
#Looking into the segments
df.groupby('RFM_level').agg({
    'recency': 'mean',
    'frequency': 'mean',
    'monetary': ['mean', 'count']
})

recency   frequency   monetary      
                       mean        mean       mean count
RFM_level                                               
Future Champion   32.007919  135.835973  30.169670  1768
High Potential   118.708869   39.213947  38.033024  1477
Low Relevance    221.483400   22.811421  -0.586430   753
SuperStar          9.973262  244.852941  45.483949   374